# Task1
Items to scrape
* Product URL
* Product Name
* Product Price
* Rating
* Number of reviews

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define constants and global variables
base_url = "https://www.amazon.in"
url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
number_of_pages = 20
temp = 0  # Counter for tracking product IDs

# Function to scrape Amazon products from a given URL
def scrape_amazon_products(url):
    global temp  # Use the global counter

    # Define request headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.amazon.in/",
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    data = []
    print("1-=-==-==-=-==-=-==")
    # Extract product details from the current page
    for product in soup.find_all("div", {"data-component-type":"s-search-result"}):
        print("2-=-==-==-=-==-=-==")

        product_url = base_url + product.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal").get("href")
        product_name = product.find("span", class_=["a-size-medium a-color-base a-text-normal","a-size-medium a-color-base a-text-bold"]).text
        product_price = float(product.find("span", class_="a-price-whole").text.replace(",", ""))
        rating=0
        try:
            rating_span = product.find("span", class_=["a-size-base", "a-icon-alt","a-size-base puis-normal-weight-text","a-size-base puis-bold-weight-text"])
            rating = float(rating_span.text.split()[0]) if rating_span else 0.0
        except ValueError:
            rating=0
        number_of_review = product.find("span", class_=["a-size-base s-underline-text","a-size-base a-color-secondary totalRatingCount"])
        if isinstance(number_of_review, str) and 'K' in number_of_review:
            try:
                number_of_review = number_of_review.text.split()[0].replace(",", "").replace("(", "").replace(")", "") if number_of_review else 0

                number_of_review = int(float(number_of_review.replace('K', '').replace('+', '')) * 1000)
            except ValueError:
                number_of_review = 0  # Set to default if unable to convert
        else:
            try:
                number_of_review = int(number_of_review)
            except ValueError:
                number_of_review = 0  # Set to default if unable to convert
            except TypeError:
                number_of_review = 0  # Set to default if unable to convert


        # Append product details to the data list
        f = [temp, product_url, product_name, product_price, rating, number_of_review]
        data.append(f)
        temp += 1

    # Find the link to the next page, if available
    pagination_container = soup.find('div', class_='s-pagination-container')
    next_url_element = pagination_container.find('a', class_='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator')
    next_url = base_url + next_url_element['href'] if next_url_element else None

    return data, next_url

# Main data collection loop
datas = []

for i in range(number_of_pages):
    data, next_url = scrape_amazon_products(url)
    if not next_url:
        print("not next url "+str(i))
        break
    url = next_url
    datas.extend(data)

# Create a pandas DataFrame from the collected data and save it to a CSV file
df = pd.DataFrame(datas, columns=["Product ID", "Product URL", "Product Name", "Product Price", "Rating", "Review Count"])
df.to_csv("amazon_products.csv", index=False)
print(df)


# Task2
With the Product URL received in the above case, hit each URL, and add below items:
* Description
* ASIN
* Product Description
* Manufacturer


In [ ]:
# Load the previously scraped data from CSV
df = pd.read_csv("amazon_products.csv")
data = []

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    print(index)
    url = row["Product URL"]
    product_id = row["Product ID"]

    # Define request headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.amazon.in/",
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    try:

        # Extract description and product description
        description = soup.find("div", id="feature-bullets").text
        prod_desc_element = soup.find("div", id="productDescription")
        prod_desc = prod_desc_element.p.span.text if prod_desc_element else ""
    
        # Extract ASIN and Manufacturer information
        prodiv = soup.find("div", "a-section feature detail-bullets-wrapper bucket")
        if prodiv is not None:
            thead = prodiv.find_all("span", "a-list-item")
            asinno = ""
            manufacturer = ""
        
            for i in range(len(thead)):
                try:
                    span_text = thead[i].find_all("span")[0].text.split()[0]
                    # print(thead[i].find_all("span"))
                    if span_text == "ASIN":
                        asinno = thead[i].find_all("span")[1].text
                    elif span_text == "Manufacturer":
                        manufacturer = thead[i].find_all("span")[1].text
                except IndexError:
                    asinno="0"
                    manufacturer="0"
            data.append([product_id, description, prod_desc, asinno, manufacturer])
    except AttributeError:
        print("AttributeError")
        
# Create a DataFrame from the collected data and merge it with the original DataFrame
df1 = pd.DataFrame(data, columns=["Product ID", "Description", "Product Description", "ASIN", "Manufacturer"])
df = df.merge(df1, on="Product ID")

# Save the merged DataFrame back to CSV
df.to_csv("amazon_products.csv", index=False)
print(df)